In [53]:
import serial #Import python serial library
from datetime import datetime
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
# You can generate an API token from the "API Tokens Tab" in the UI
token = "ks_nSY132I3RRIS6y3JITJRtF4RckdpHqZwN8dmNJG7iFML9Kk89kT_A2qDzdGKU7DVLnPsKTnbWmZiDhSyzgg=="
org = "UQTR"
bucket = "Battery"

influxClient= InfluxDBClient(url="http://localhost:8086", token=token, org=org)
write_api = influxClient.write_api(write_options=SYNCHRONOUS)


#serialPort = serial.Serial("COM3",9600,timeout=1,parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS) #open Serial Port
serialPort = serial.Serial("/dev/ttyACM0",9600,timeout=1,parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS) #Ubuntu open Serial Port
i=0
print(datetime.utcnow())
Continue=True;
while Continue:
    try:
        #Read a single ine from serial (Message that ends by LF \n)
        data=serialPort.readline()
        dataString = data.replace(b'\x00', b'').decode('ascii') #Delete null characters
        if data==0x00:
            print("No Data")
        else:
            #Process and Transform data from MCU
            timestamp=datetime.utcnow()
            i=i+1
            dataList=dataString.split(",")
            currentInfo=dataList[0].split("=")
            voltageInfo=dataList[1].split("=")
            if(currentInfo[0]=="Current"):
                currentValue=float(currentInfo[1])
            if(voltageInfo[0]== "Voltage"):
                voltageValue=float(voltageInfo[1])
            #Save Time Series to Influxdb
            point = Point("PowerMeasurement")\
            .field("current", currentValue)\
            .field("voltage",voltageValue)\
            .time(timestamp,write_precision='ns')
            print(datetime.utcnow())
            write_api.write(bucket, org, point)
    except KeyboardInterrupt:
        print("Stop requested by User")
        Continue=False
    except: 
        print("An Error has occurred... Sending Previous Data")
        point = Point("PowerMeasurement")\
        .field("current", currentValue)\
        .field("voltage",voltageValue)\
        .time(timestamp,write_precision='ns')
        print(datetime.utcnow())
        write_api.write(bucket, org, point)
            
            
print(datetime.utcnow())     
serialPort.close()
print("Port Closed")
influxClient.close()
print("client Close")




1643320691.096423

In [ ]:
import serial #Import python serial library
import time
import re
controllerSerial = serial.Serial("/dev/ttyUSB0",115200,timeout=10e-3,parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS);

controllerSerial.write("v\r".encode())
speed=controllerSerial.read_until('\r\n>')
speedString = speed.replace(b'\x00', b'').decode('ascii') #Delete null characters
speedData=speedString.split("\n")[1]
speedData=speedData.replace("\r","").replace("\n","") #Remove \r\n
velocity=re.split("L:|R:",speedData)
print(velocity)
speedL=(int(velocity[1]))/20e-3*60/(1000*2) #v/Lr*60/(ppr*2)
speedR=(int(velocity[2]))/20e-3*60/(1000*2)




In [ ]:
import serial
import time
import re
controllerSerial = serial.Serial("/dev/ttyUSB0",115200,timeout=10e-3,parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS);
speedData=speedMeasurement(controllerSerial)
print(speedData[0],speedData[1])




In [ ]:
import serial #Import python serial library
from datetime import datetime
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from RobotMeasures import powerMeasurement, speedMeasurement, sendData
# You can generate an API token from the "API Tokens Tab" in the UI
token = "ks_nSY132I3RRIS6y3JITJRtF4RckdpHqZwN8dmNJG7iFML9Kk89kT_A2qDzdGKU7DVLnPsKTnbWmZiDhSyzgg=="
org = "UQTR"

influxClient= InfluxDBClient(url="http://localhost:8086", token=token, org=org)
write_api = influxClient.write_api(write_options=SYNCHRONOUS)
serialPort = serial.Serial("/dev/ttyACM0",9600,timeout=1,parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS) #MCU open Serial Port
controllerSerial = serial.Serial("/dev/ttyUSB0",115200,timeout=10e-3,parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS); #AF160 Serial Port
i=0
print(datetime.utcnow())
Continue=True;
while Continue:
    try:
        #Read a single ine from serial (Message that ends by LF \n)
        data=serialPort.readline()
        dataString = data.replace(b'\x00', b'').decode('ascii') #Delete null characters
        if data==0x00:
            print("No Data")
        else:
            #Process and Transform data from MCU
            timestamp=datetime.utcnow()
            i=i+1
            #get speed Data
            [speedL,speedR]=speedMeasurement(controllerSerial)
            [voltageValue,currentValue]=powerMeasurement(dataString)
            #Save Time Series to Influxdb
            sendData(write_api,currentValue,voltageValue,speedL,speedR,timestamp)
    except KeyboardInterrupt:
        print("Stop requested by User")
        Continue=False
    except: 
        print("An Error has occurred... Sending Previous Data")
        sendData(write_api,currentValue,voltageValue,speedL,speedR,timestamp)
            
            
print(datetime.utcnow())     
serialPort.close()
print("Port Closed")
influxClient.close()
print("client Close")


#functions

import serial #Import python serial library
from datetime import datetime
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import re

def powerMeasurement(dataString):
    dataList=dataString.split(",")
    currentInfo=dataList[0].split("=")
    voltageInfo=dataList[1].split("=")
    if(currentInfo[0]=="Current"):
        currentValue=float(currentInfo[1])
    if(voltageInfo[0]== "Voltage"):
        voltageValue=float(voltageInfo[1])
    return [voltageValue,currentValue]


def speedMeasurement(controllerSerial):
    controllerSerial.write("v\r".encode())
    speed=controllerSerial.read_until('\r\n>')
    speedString = speed.replace(b'\x00', b'').decode('ascii') #Delete null characters
    speedData=speedString.split("\n")[1]
    speedData=speedData.replace("\r","").replace("\n","") #Remove \r\n
    velocity=re.split("L:|R:",speedData)
    print(velocity)
    speedL=(int(velocity[1]))/20e-3*60/(1000*2) #v/Lr*60/(ppr*2)
    speedR=(int(velocity[2]))/20e-3*60/(1000*2)
    return [speedL,speedR]

def sendData(write_api, bucket,currentValue,voltageValue,speedL,speedR,timestamp):
    point = Point("RobotMeasures")\
    .field("current", currentValue)\
    .field("voltage",voltageValue)\
    .field("speedL",speedL)\
    .field("speedR",speedR)\
    .time(timestamp,write_precision='ns')
    print(datetime.utcnow())
    write_api.write(bucket, org, point)